In [ ]:
!git clone https://github.com/retasu0/sokudoku_demo.git
!pip install mecab-python3
!pip install unidic
!python -m unidic download

In [ ]:
from tools import *
import cv2
import glob
import os
import shutil
import os
os.chdir("sokudoku_demo")

In [ ]:
# ---------------
# 各種パラメータ
# ---------------
MAX_LEN = 7 # 1つのコマで表示する文字数の上限
WIDTH = 1200 # 動画の解像度。横幅
HEIGHT = 600 # 動画の解像度。立幅
FRAME_SPEED = 5 # 1秒間に何枚のコマを表示するか。小数も可能

# ターゲットの文章
text = """
マイクロソフト（英: Microsoft Corporation）は、アメリカ合衆国ワシントン州に本社を置く、ソフトウェアを開発、販売する会社である。2021年11月時点での時価総額が約270兆円を超え世界2位[3]。1975年にビル・ゲイツとポール・アレンによって創業された[注釈 1]。1985年にパソコン用OSのWindowsを開発。1990年にWindows向けのオフィスソフトとしてMicrosoft Officeを販売。1995年にウェブブラウザのInternet Explorerをリリース。2001年に家庭用ゲーム機のXboxを販売。2009年に検索エンジンのBingを設立。2010年にクラウドサービスとしてAzureを開始。
"""
# ---------------
# 文章の加工
# ---------------

# 文節ごとに分ける
text_list = bunsetsuWakachi(text)


# 文節の上限文字数を超えないようにまとめたリスト
sentence_list = []

tmp_str = ""
for text in text_list:

    # 指定文字数よりも小さければ追加
    if len(tmp_str) + len(text)<MAX_LEN:
        tmp_str+=text
    # 指定文字数よりも大きければリストに追加、リセット
    else:
        sentence_list.append(tmp_str)
        tmp_str=text

sentence_list.append(tmp_str)

# ---------------
# 画像の生成
# ---------------

# 動画に使う画像を生成
for index,text in enumerate(sentence_list):
    make_image(text,index)


# ---------------
# 動画の作成
# ---------------

size=(WIDTH,HEIGHT)#サイズ指定
fourcc = cv2.VideoWriter_fourcc('m','p','4','v')#保存形式
save = cv2.VideoWriter('output.mp4',fourcc,4,size)#動画を保存する形を作成


#0フォルダ
pic_data=glob.glob("data/tmp_img/*.png")
for i in range(len(pic_data)):
    img=pic_data[i]
    img=cv2.imread(img)#画像を読み込む
    img=cv2.resize(img,(WIDTH,HEIGHT))
    save.write(img)#保存


save.release() # ファイルを閉じる

# ---------------
# 画像の削除
# ---------------
shutil.rmtree("data/tmp_img/")
os.mkdir("data/tmp_img/")


# ---------------
# mp4動画の再生
# ---------------

from IPython.display import HTML
from base64 import b64encode

mp4 = open('./sample.mp4', 'rb').read()
data_url = 'data:video/mp4;base64,' + b64encode(mp4).decode()
HTML(f"""
<video width="100%" height="100%" controls>
      <source src="{data_url}" type="video/mp4">
</video>""")
